<a href="https://colab.research.google.com/github/tamrazebian/pylookml/blob/main/pylookML_for_community.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate into your Looker environment with your Looker instance URL and API Client ID and Secret
base_url = ''
client_id=''
client_secret=''

contents = """
[Looker]
# Base URL for API. Do not include /api/* in the url
base_url = {}
# API 3 client id
client_id = {}
# API 3 client secret
client_secret = {}
# Set to false if testing locally against self-signed certs. Otherwise leave True
""".format(base_url,client_id,client_secret)

with open ("api.ini", "w") as f:
  f.write(contents)

In [ ]:
# You might need to manually download these 2 python library files and drag them into colab: https://pypi.org/project/lookml/2.0.0/#files
!python3 -m pip install ./lookml-2.0.0.tar.gz
!python3 -m pip install ./lookml-2.0.0-py3-none-any.whl
!pip install lookml

Processing ./lookml-2.0.0.tar.gz
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.7 MB/s eta 0:00:00
  Created wheel for lookml: filename=lookml-2.0.0-py3-none-any.whl size=19393 sha256=64d5bb652349121d75a61084c79c6ba39faeff8b9d048fb8ae394cc49759508b
  Stored in directory: /root/.cache/pip/wheels/d2/2f/5c/9c93e9abc97b3d82729b8c0213a7fbc1caca55f8aee20cb3cb
Successfully built lookml
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
Processing ./lookml-2.0.0-py3-none-any.whl
lookml is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
!pip install looker_sdk
import json
import lookml
import looker_sdk
from looker_sdk import methods40, models40, init40
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.5/226.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
# Step 1 -- connect to the Looker API to pull a list of EAV fields
 # SDK = init31("api.ini") Updated to new API 4.0
sdk = looker_sdk.init40("api.ini")
 # Modify the below to get our appropriate fields
sql_for_fields = f"""
    SELECT  user_id
            ,field
            ,value
    FROM `tz-looker-core-argolis.ce_tech_talk.pylookml`
 """
query_config = models40.SqlQueryCreate(sql=sql_for_fields, connection_name="sample_bigquery_connection")
query = sdk.create_sql_query(query_config)
response = json.loads(sdk.run_sql_query(slug=query.slug, result_format="json"))
print(response)

[{'user_id': 1, 'field': 'age', 'value': '20'}, {'user_id': 4, 'field': 'age', 'value': '25'}, {'user_id': 2, 'field': 'age', 'value': '30'}, {'user_id': 3, 'field': 'age', 'value': '40'}, {'user_id': 4, 'field': 'height', 'value': '170'}, {'user_id': 3, 'field': 'height', 'value': '175'}, {'user_id': 2, 'field': 'height', 'value': '180'}, {'user_id': 1, 'field': 'height', 'value': '183'}, {'user_id': 1, 'field': 'name', 'value': 'Bob'}, {'user_id': 3, 'field': 'name', 'value': 'Sam'}, {'user_id': 4, 'field': 'name', 'value': 'Jane'}, {'user_id': 2, 'field': 'name', 'value': 'John'}, {'user_id': 1, 'field': 'eye_color', 'value': 'blue'}, {'user_id': 2, 'field': 'eye_color', 'value': 'brown'}, {'user_id': 4, 'field': 'eye_color', 'value': 'brown'}, {'user_id': 3, 'field': 'eye_color', 'value': 'green'}]


In [ ]:
 # Step 2 -- Create a pyLookML project connection to your github
 proj = lookml.Project(
         # The github location of the repo
          repo = ''
         # git_url= '[yourgit]]'
         # Instructions on creating an access token: https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line
         ,access_token=''
         # Your Looker host
         ,looker_host=''
         # The name of the project on your looker host
         ,looker_project_name=''
         # You can deploy to branches other than master, a shared or personal branch if you would like to hop into looker, pull
         # Remote changes and review before the code is committed to production
         ,branch=''
        #  ,outputPath =' .tmp'
 )
 # For simplicity of this example, all of the objects we're tracking will be contained in the model file, but can be split across the project based on your needs.
 modelFile = proj['/pylookml.model.lkml']

In [ ]:
print (modelFile)

connection: "sample_bigquery_connection"

 explore: flat { 
   
   }


 explore: _eav_flattener { 
  from: pylookml
  hidden: yes 
   }


view: pylookml {
  sql_table_name: `tz-looker-core-argolis.ce_tech_talk.pylookml` ;;
  
  
  dimension: field {   type: string
    sql: ${TABLE}.field ;; }
  dimension: user_id {   type: number
    sql: ${TABLE}.user_id ;; }
  dimension: value {   type: string
    sql: ${TABLE}.value ;; }
  
  measure: age {   type: max
    sql: CASE WHEN ${field} = 'age' THEN ${value} ELSE NULL END ;; }
  measure: eye_color {   type: max
    sql: CASE WHEN ${field} = 'eye_color' THEN ${value} ELSE NULL END ;; }
  measure: height {   type: max
    sql: CASE WHEN ${field} = 'height' THEN ${value} ELSE NULL END ;; }
  measure: name {   type: max
    sql: CASE WHEN ${field} = 'name' THEN ${value} ELSE NULL END ;; }
  }


view: flat {
  derived_table: { explore_source: _eav_flattener { 
    column: user_id {     field: _eav_flattener.user_id }
    column: eye_color {    

In [ ]:
# Step 3 -- Set up the objects we'll be manipulating
eavSource = modelFile['views']['pylookml']
# The user profile we'll call the "flattening NDT" since that's where our flattening logic lives
flatteningNDT = modelFile['views']['flat']
# Ensure there is a hidden explore to expose the eav_souce transformations to our user_profile NDT
modelFile + f'''
     explore: _eav_flattener {{
         from: {eavSource.name}
         hidden: yes
     }}
 '''
# Begin the derived table, will be added to as we loop through the fields
drivedtableString = f'''
     derived_table: {{
         explore_source: _eav_flattener {{
             column: user_id {{ field: _eav_flattener.user_id }}
 '''

# Set up a pair of lists to track the unique org IDs and column names
# Since the API query will be at a org/column level, this allows us to "de-dupe"
orgIds, columns = [], []

In [ ]:
# orgIDs are User IDs in our example
# Step 4
for column in response:
     dimName = lookml.lookCase(column['field'])
     orgIds.append(column['user_id'])
     columns.append(dimName)
     # Step 1) Add flattening measure to the EAV source table
     eavSource + f'''
             measure: {dimName} {{
                 type: max
                 sql: CASE WHEN ${{field}} = '{column['field']}' THEN ${{value}} ELSE NULL END;;
             }}
     '''

     # Add to the NDT fields
     flatteningNDT + f'''
             dimension: {dimName} {{
                 label: "{dimName}"
                 type: string
                 sql: ${{TABLE}}.{dimName} ;;
             }}
     '''
     # If column['LOOKER_TYPE'] == "measureA":
     if re.search('Measure.+', column['field']):
         # Print({dimName}) @James, added cast to measure!
         flatteningNDT + f'''
             measure: {dimName}_total {{
                 label: "{dimName}_total"
                 type: sum
                 sql: CAST(${{{dimName}}} AS NUMERIC) ;;
             }}
         '''

In [ ]:
# Step 5
for col in set(columns):
    drivedtableString += f'''
      column: {col} {{ field: _eav_flattener.{col} }}
    '''
drivedtableString += '}}'

In [ ]:
 # Step 6 -- Add all the final objects back to the model file
 # Finish by adding some of the strings we've been tracking:
 flatteningNDT + drivedtableString
 # Add access grants to the model
 modelFile

In [ ]:
 # Step 8 save to github
 proj.put(modelFile)

In [ ]:
# Step 9 push from github to looker. You might need to refresh Looker before you can pull down the changes
proj.deploy()